# Selenium Basic command

1. driver.get('url') : 페이지 접속
2. elem = driver.find_element_by_name('q') : 커서를 검색어 입력 부분에 위치시킴
    - find_by_class_name or find_by_xpath 등도 가능 (xpath는 html 검사 시 우클릭>xpath copy로 복사 가능
3. elem.clear() : 검색어 부분 입력내용 삭제
4. elem.send_keys('내용') : 내용 입력
5. elem.submit() : 입력한 내용 검색 실행
    - click() : 클릭
6. driver.close() : 페이지 닫음
7. aseert 'keyword' in driver.title or driver.page_source : 제목이나 내용에 keyword가 있는지 파악 - 없으면 assertion error 반환

In [161]:
import pandas as pd
import numpy as np

import selenium
from selenium import webdriver

from bs4 import BeautifulSoup as bs
from urllib.request import urlopen, Request
from urllib.parse import quote_plus

import time
from tqdm import tqdm_notebook
import warnings
warnings.filterwarnings('ignore')

In [ ]:
def instagram_login(id, pw):
    driver.get(url)
    driver.implicitly_wait(5)
    driver.find_element_by_name('username').send_keys(id)    # id 입력
    elem_pw = driver.find_element_by_name('password')    # pw 입력
    elem_pw.send_keys(pw)
    elem_pw.submit()
    
    driver.implicitly_wait(5)    # 파싱될 때까지 5초 기다림 (미리 완료되면 waiting 종료됨)
    driver.find_element_by_class_name('cmbtv').click()    # 비밀번호 저장하지 않음
    
    driver.implicitly_wait(5)
    driver.find_element_by_xpath('/html/body/div[4]/div/div/div/div[3]/button[2]').click()    # 알림설정 무시

In [ ]:
def main_search(keyword):
    search = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/input')
    search.clear()
    search.send_keys(keyword)
    search_list1 = driver.find_element_by_xpath('//*[@id="react-root"]/section/nav/div[2]/div/div/div[2]/div[3]/div[2]/div/a[1]')
    search_list1.click()

In [277]:
#접속주소 설정
url = 'http://www.instagram.com'
path = 'C:\\Users\\rectworks\\Downloads\\chromedriver_win32\\chromedriver.exe'
driver = webdriver.Chrome(path)

In [278]:
instagram_login("USER_ID", "PASSWORD")

In [286]:
main_search('KEYWORD')

첫 번째 게시글 클릭
driver.find_element_by_xpath('/html/body/div[1]/section/main/article/div[1]/div/div/div[1]/div[1]/a/div').click()

In [287]:
SCROLL_PAUSE_TIME = 2.0
post_link = []

while True:
    pageString = driver.page_source   # page_source : 현재 렌더링된 페이지의 Elements를 모두 가져옴
    bsObj = bs(pageString, 'lxml')

    for postline in bsObj.find_all(name='div', attrs={"class":"Nnq7C weEfm"}):
        for post in range(3):   # 인스타그램 게시글은 행별로 3개의 게시물로 나타나기 때문에 range 3으로 지정
            item = postline.select('a')[post]
            link = item.attrs['href']
            post_link.append(link)

    last_height = driver.execute_script('return document.body.scrollHeight')   # 자바스크림트로 스크롤 길이를 넘겨줌
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")   # selenium에서 scroll 기능 사용
    time.sleep(SCROLL_PAUSE_TIME)
    # 프로세스 자체를 지정시간동안 기다려줌(무조건 지연)
    #driver.implicitly_wait(SCROLL_PAUSE_TIME)
    # 브라우저 엔진에서 파싱되는 시간을 기다려줌(요소가 존재하면 지연없이 코드 실행)
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(SCROLL_PAUSE_TIME)
        #driver.implicitly_wait(SCROLL_PAUSE_TIME)
        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        else:
            last_height = new_height
            continue

KeyboardInterrupt: 

In [288]:
num_of_post = len(post_link)

print('총 {0}개의 게시글을 수집합니다.'.format(num_of_post))

id_list = []
like_list = []
tag_list = []
link_list = []

for i in tqdm_notebook(range(num_of_post)):
    
    tag_list.append([])
    #req = Request("https://www.instagram.com/p"+reallink[i], headers={'User-Agent': 'Mozila/5.0'})
    req = Request("https://www.instagram.com" + post_link[i], headers={'User-Agent': 'Mozila/5.0'})
    link_list.append("https://www.instagram.com" + post_link[i])

    postpage = urlopen(req).read()
    post_body = bs(postpage, 'lxml', from_encoding='utf-8')
    post_core = post_body.find('meta', attrs={'property':"og:description"})
    contents = post_core['content']
    
    try:
        likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
    except:
        likes=0
        
    if "@" and ")" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find(")")]
    elif "@" and ")" not in contents and "on Instagram" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
    else :
        personal_id = contents[1: contents.find(' posted on')]
        

    #personal_id = contents[contents.find("@")+1 : contents.find(")")]# (@ personal_id) on Instagram 형식이므로 @,) 기준으로 계정 추출
    
    #if personal_id == '':
    #    personal_id = "Null"  # 아이디 없으면 Null 입력 ( Null이 있을 수 있나? )
        
    id_list.append(personal_id)
    like_list.append(likes)

    for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
        hashtags = tag_content['content'].rstrip(',')
        tag_list[i].append(hashtags)

총 2484개의 게시글을 수집합니다.


In [289]:
insta_dict = {'계정':id_list,
             '좋아요':like_list,
             '해시태그':tag_list,
             '링크':link_list}
df = pd.DataFrame(insta_dict)
df

,계정,좋아요,해시태그,링크
0,hn._.zu,590,[],https://www.instagram.com/p/CC1-KMyHphG/
1,binyi22,305,"[selca, 일상, 커플룩, 좋아요, 소통, 데일리, 좋반, 좋아요테러, 데이트,...",https://www.instagram.com/p/CC06YUJJ__S/
2,byuneunzi,330,"[평생, 나랑놀자]",https://www.instagram.com/p/CCyJsJqjJ0H/
3,hyunjinvely,280,[],https://www.instagram.com/p/CC2sOe2grVC/
4,nikaakz,800,[],https://www.instagram.com/p/CC1R5_gF-zA/
...,...,...,...,...
2479,goldennailcc,5,"[거두리, 로데오, 네일아트, 네일, 춘천네일샵, 골든네일, 석사동, 네일샵, na...",https://www.instagram.com/p/CC3fljEDxxI/
2480,dda_eunxx,83,"[비가안와서다행, 춘천, 호호]",https://www.instagram.com/p/CC3fghhDYh9/
2481,staring0601 posted on their Instagram profile:...,0,"[근화동맛집, 춘천스카이워크맛집, 춘천모임장소, 선팔, 맞팔해요, 춘천드라이브, 소...",https://www.instagram.com/p/CC3fPGBlbAZ/
2482,cieldelime,8,"[맛집, 미스사이공, 춘천, 라이스치킨텐더, 쌀국수, 회사창립기념일]",https://www.instagram.com/p/CC3fNWLHtJ2/


In [264]:
req = Request("https://www.instagram.com" + post_link[27], headers={'User-Agent': 'Mozila/5.0'})
postpage = urlopen(req).read()
post_body = bs(postpage, 'lxml', from_encoding='utf-8')
post_core = post_body.find('meta', attrs={'property':"og:description"})
print(post_core)
contents = post_core['content']
print(contents)

<meta content="@fly_heni posted on their Instagram profile: “#bali#seminyak#alila#infinitypool#발리#인도네시아#스미냑#여기오늘 내가 전세냇다 🌴🙋🏻💕😘 발리는 비수기에 올것 그리고 모기약 가져올것😂😂 this…”" property="og:description"/>
@fly_heni posted on their Instagram profile: “#bali#seminyak#alila#infinitypool#발리#인도네시아#스미냑#여기오늘 내가 전세냇다 🌴🙋🏻💕😘 발리는 비수기에 올것 그리고 모기약 가져올것😂😂 this…”


In [272]:
num_of_post = len(post_link)

print('총 {0}개의 게시글을 수집합니다.'.format(num_of_post))

id_list = []
like_list = []
tag_list = []
link_list = []

for i in tqdm_notebook(range(num_of_post)):
    
    tag_list.append([])
    #req = Request("https://www.instagram.com/p"+reallink[i], headers={'User-Agent': 'Mozila/5.0'})
    req = Request("https://www.instagram.com" + post_link[i], headers={'User-Agent': 'Mozila/5.0'})
    link_list.append("https://www.instagram.com" + post_link[i])

    postpage = urlopen(req).read()
    post_body = bs(postpage, 'lxml', from_encoding='utf-8')
    post_core = post_body.find('meta', attrs={'property':"og:description"})
    contents = post_core['content']
    
    try:
        likes = int(contents[: contents.find(' Likes, ')])  # Likes 문자열 앞에 있는 좋아요 개수 추출
    except:
        likes=0
        
    if "@" and ")" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find(")")]
    elif "@" and ")" not in contents and "on Instagram" in contents:
        personal_id = contents[contents.find("@")+1 : contents.find('on Instagram')]
    else :
        personal_id = contents[1: contents.find(' posted on')]
        

    #personal_id = contents[contents.find("@")+1 : contents.find(")")]# (@ personal_id) on Instagram 형식이므로 @,) 기준으로 계정 추출
    
    #if personal_id == '':
    #    personal_id = "Null"  # 아이디 없으면 Null 입력 ( Null이 있을 수 있나? )
        
    id_list.append(personal_id)
    like_list.append(likes)

    for tag_content in post_body.find_all('meta', attrs={'property':"instapp:hashtags"}):
        hashtags = tag_content['content'].rstrip(',')
        tag_list[i].append(hashtags)

#driver.close()

총 30개의 게시글을 수집합니다.


In [275]:
df = pd.DataFrame(data=dict(id_list),dict(like_list),dict(tag_list),dict(link_list), columns=['계정','좋아요','해시태그','링크'])
df

SyntaxError: positional argument follows keyword argument (<ipython-input-275-60b516b488bf>, line 1)

In [276]:
insta_dict = {'계정':id_list,
             '좋아요':like_list,
             '해시태그':tag_list,
             '링크':link_list}
df = pd.DataFrame(insta_dict)
df

,계정,좋아요,해시태그,링크
0,cc_coopcenter,13,"[네이쳐앤드피플, 홈페이지, 여기오늘, www, 춘천시협동조합지원센터]",https://www.instagram.com/p/B9YbBjupqfI/
1,unique.h.s,6,"[여기오늘, 밤산책]",https://www.instagram.com/p/Bb9M-Z6F_Do/
2,s.hyeyoung,52,"[흔들리지않고피는꽃이어디있으랴, 여기오늘]",https://www.instagram.com/p/B9blFm8nAnk/
3,s.hyeyoung,42,"[일상, 서시천산책로, 섬진강줄기따라, 노고단은멀리서구경😁, 슬기로운연차생활, 나도...",https://www.instagram.com/p/CBP7ou_nRdU/
4,s.hyeyoung,44,[],https://www.instagram.com/p/CAtyeJBn90_/
5,s.hyeyoung,52,"[산청, 힐링여행]",https://www.instagram.com/p/CBaFfBwnPO4/
6,_mongureum,166,"[똑구야😖😖, 쌈박하게뛰고들가쟈구나, 모기천국이로구나, 여기오늘]",https://www.instagram.com/p/BIPs8ZjDeLP/
7,ayeoni_88,34,"[방송탔다, 여기오늘, 전통시장축제, 파래김, 노엠에스쥐, 기가마킴, 기가맛김, 양...",https://www.instagram.com/p/BZe_RggAkrM/
8,hong.pa,71,"[유쟁네로, 여기오늘, 우리가, 술달리러, 힘들구만, 중학교모임, 월드컵보러, 내일...",https://www.instagram.com/p/BkXokh9lMF9/
9,untersbergbahn,26,[],https://www.instagram.com/p/B91zZ4BnIk0/


In [67]:
soup1

<meta content="13 Likes, 0 Comments - 춘천시협동조합지원센터 (@cc_coopcenter) on Instagram: “춘천시협동조합지원센터  홈페이지가 개설되었습니다 ;) 센터 교육 및 사업 정보를 확인해 보세요  제작에 힘써주신 사회적기업 #네이쳐앤드피플 예비사회적기업 #여기오늘 감사드려요…”" property="og:description"/>

In [123]:
a = soup1['content']
a

'13 Likes, 0 Comments - 춘천시협동조합지원센터 (@cc_coopcenter) on Instagram: “춘천시협동조합지원센터  홈페이지가 개설되었습니다 ;) 센터 교육 및 사업 정보를 확인해 보세요  제작에 힘써주신 사회적기업 #네이쳐앤드피플 예비사회적기업 #여기오늘 감사드려요…”'

In [138]:
like = a[: a.find('Likes, ')]
int(like)

13

In [136]:
person_id = a[a.find("@") + 1 : a.find(")")]
person_id

'cc_coopcenter'

In [180]:
for tag_content in soup.find_all('meta', attrs={'property':"instapp:hashtags"}):
    print(tag_content)
    print('1213')

<meta content="네이쳐앤드피플" property="instapp:hashtags"/>
1213
<meta content="여기오늘" property="instapp:hashtags"/>
1213
<meta content="홈페이지" property="instapp:hashtags"/>
1213
<meta content="춘천시협동조합지원센터" property="instapp:hashtags"/>
1213
<meta content="www" property="instapp:hashtags"/>
1213


In [ ]:
for i in len(csvtext):
    csvtext_word = 

In [49]:
set(csvtext)

TypeError: unhashable type: 'list'

In [115]:
a = '(@아이디) ":)"'
a[a.find('@')+1 : a.find(')')]
# find : index 값 반환 ,  처음 나오는 문자를 기준으로 함

'아이디'

In [94]:
csvtext

[['cc_coopcenter', 'www', '홈페이지', '여기오늘', '춘천시협동조합지원센터', '네이쳐앤드피플'],
 ['unique.h.s', '밤산책', '여기오늘'],
 ['s.hyeyoung', '흔들리지않고피는꽃이어디있으랴', '여기오늘'],
 ['s.hyeyoung',
  '감성산책',
  '서시천산책로',
  '산책로',
  '노고단은멀리서구경😁',
  '섬진강줄기따라',
  '신돌콩',
  '여기행복',
  '여행',
  '대한민국구석구석',
  '캐논',
  '감성한스푼',
  '카메라',
  '전남구례',
  '일상',
  '나도구례💕',
  '슬기로운연차생활',
  '여기오늘',
  '지리산',
  '노을찾아'],
 ['s.hyeyoung'],
 ['s.hyeyoung', '힐링여행', '산청'],
 ['_mongureum', '똑구야😖😖', '여기오늘', '모기천국이로구나', '쌈박하게뛰고들가쟈구나'],
 ['ayeoni_88',
  '4번출구',
  '파래김',
  '기가마킴',
  '맛스타그램',
  '노엠에스쥐',
  '명품김',
  '전통시장축제',
  '시노시작🖖',
  '서울양재at센터',
  '양재시민의숲역',
  '여기오늘',
  '전통시장',
  '핫해하태🔥',
  '놀러오세요💕',
  '재래김',
  '양재at센터',
  '서울경기',
  '여섯시내고향',
  '방송탔다',
  '기가맛김'],
 ['hong.pa',
  '동중',
  '내일은없구나',
  '장원가든',
  '월드컵보러',
  '맛집',
  '2차는',
  '전세냄',
  '여기오늘',
  '중학교모임',
  '술달리러',
  '유쟁네로',
  'dsm',
  '힘들구만',
  '가즈아',
  '우리가',
  '화이팅',
  '가족'],
 ['untersbergbahn on In'],
 ['eun_kyung_sin',
  '태양이네',
  '월정리해변',
  '편안해질',
  '게스트하우스',
  '여기오늘',
  '애기데리고',
  '태양이장난감'